In [ ]:
from symbolfit.symbolfit import *

# Dataset

Five inputs are needed, which can be python lists or numpy arrays:
1. ``x``: independent variable (bin center values).
2. ``y``: dependent variable.
3. ``y_up``: upward uncertainty in y per bin.
4. ``y_down``: downward uncertainty in y per bin.
5. ``bin_widths_1d`` bin widths for x.

- Elements in both y_up and y_down should be non-negative values.
- These values are the "delta" in y,
  - y + y_up = y shifted up by one standard deviation.
  - y - y_down = y shifted down by one standard deviation.
- If no uncertainty in the dataset, one can set y_up and y_down to ones with the same shape as x.

In [ ]:
x = [12.5, 37.5, 62.5, 87.5, 112.5, 137.5, 162.5, 187.5, 212.5, 237.5, 262.5, 287.5, 312.5, 337.5, 362.5, 387.5, 412.5, 437.5, 462.5, 487.5]
y = [10.234884262084961, 122.1119384765625, 338.9125061035156, 810.2549438476562, 649.0571899414062, 351.8170166015625, 248.619873046875, 186.88763427734375, 141.754150390625, 103.42931365966797, 78.36450958251953, 60.3994255065918, 49.005863189697266, 33.54744338989258, 27.76025390625, 25.299283981323242, 19.729631423950195, 14.033162117004395, 15.06820011138916, 9.641764640808105]
y_up = [3.199200566092248, 11.050427072134475, 18.409576478113657, 28.464977495997715, 25.476600831771226, 18.756785881423355, 15.767684454189048, 13.670685216087149, 11.906055198537633, 10.170020337229811, 8.852373104570296, 7.771706730608908, 7.000418786736781, 5.7920154859852175, 5.268800044246317, 5.029839359395411, 4.441804973650936, 3.746086239931536, 3.8817779575072504, 3.105119102515732]
y_down = [3.199200566092248, 11.050427072134475, 18.409576478113657, 28.464977495997715, 25.476600831771226, 18.756785881423355, 15.767684454189048, 13.670685216087149, 11.906055198537633, 10.170020337229811, 8.852373104570296, 7.771706730608908, 7.000418786736781, 5.7920154859852175, 5.268800044246317, 5.029839359395411, 4.441804973650936, 3.746086239931536, 3.8817779575072504, 3.105119102515732]
bin_widths_1d = [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]

Plot the dataset to see what we will be fitting to:

In [ ]:
fig, axes = plt.subplots(figsize = (6, 4))
plt.errorbar(np.array(x).flatten(),
             np.array(y).flatten(),
             yerr = [np.array(y_down).flatten(), np.array(y_up).flatten()],
             xerr = np.array(bin_widths_1d)/2,
             fmt = '.', c = 'black', ecolor = 'grey', capsize = 0,
            )

# Configure the fit

## Configure PySR to define the function space being searched for with symbolic regression

In [ ]:
from pysr import PySRRegressor
import sympy

pysr_config = PySRRegressor(
    model_selection = 'accuracy',
    niterations = 200,
    maxsize = 60,
    binary_operators = [
        '+', '*'
                     ],
    unary_operators = [
        'exp',
        'gauss(x) = exp(-x*x)',
        'tanh',
    ],
    nested_constraints = {
        'tanh':   {'tanh': 0, 'exp': 0, 'gauss': 0, '*': 2},
        'exp':    {'tanh': 0, 'exp': 0, 'gauss': 0, '*': 2},
        'gauss':  {'tanh': 0, 'exp': 0, 'gauss': 0, '*': 2},
        '*':      {'tanh': 1, 'exp': 1, 'gauss': 1, '*': 2},
    },
    extra_sympy_mappings={
        'gauss': lambda x: sympy.exp(-x*x),
                         },
    loss='loss(y, y_pred, weights) = (y - y_pred)^2 * weights',
)

Here, we allow two binary operators (+, *) and three unary operators (exp, gauss, tanh) when searching for functional forms.

Nested constraints are imposed to prohibit, e.g., exp(exp(x))...

Loss function is a weighted MSE, where the weight is the sqaured uncertainty by default in SymbolFit.

For PySR options, please see:
- https://github.com/MilesCranmer/PySR
- https://astroautomata.com/PySR/

## Configure SymbolFit with the PySR config and for the re-optimization process

In [ ]:
model = SymbolFit(
        # Dataset: x, y, y_up, y_down.
    	x = x,
    	y = y,
    	y_up = y_up,
    	y_down = y_down,
        # PySR configuration of function space.
    	pysr_config = pysr_config,
        # Constrain the maximum function size and over-write maxsize in pysr_config.
    	max_complexity = 60,
        # Whether to scale input x to be within 0 and 1 during fits for stability, as large x could lead to overflow.
    	input_rescale = True,
        # Whether to scale y during fits for stability (when input_rescale is True): None / 'mean' / 'max' / 'l2'.
    	scale_y_by = 'mean',
        # Set a maximum standard error (%) for all parameters to avoid bad fits during re-optimization (will re-parameterize and re-fit with fewer parameters when too large errors).
    	max_stderr = 20,
        # Consider y_up and y_down to weight the MSE loss during SR search and re-optimization.
    	fit_y_unc = True,
        # Set a random seed for returning the same batch of functional forms every time (single-threaded), otherwise set None to explore more functions every time.
    	random_seed = None,
        # Custome loss weight to replace y_up and y_down.
    	loss_weights = None
)

# Symbol Fit it!

Run the fit: SR fit for functional forms -> parameterization -> re-optimization fit for improved best-fits and uncertainty estimation -> evaluation.

In [ ]:
model.fit()

## Save results to output files

Save results to csv tables:
- ``candidates.csv``: saves all candidate functions and evaluations in a csv table.
- ``candidates_reduced.csv``: saves a reduced version for essential information without intermediate results.

In [ ]:
model.save_to_csv(output_dir = 'output_dir/')

Plot results to pdf files:
- ``candidates.pdf``: plots all candidate functions with associated uncertainties one by one for fit quality evaluation.
- ``candidates_sampling.pdf``: plots all candidate functions with total uncertainty coverage generated by sampling parameters.
- ``candidates_gof.pdf``: plots the goodness-of-fit scores.
- ``candidates_correlation.pdf``: plots the correlation matrices for the parameters of the candidate functions.

In [ ]:
model.plot_to_pdf(
    	output_dir = 'output_dir/',
    	bin_widths_1d = bin_widths_1d,
    	#bin_edges_2d = bin_edges_2d,
    	plot_logy = False,
    	plot_logx = False,
        sampling_95quantile = False
)